#### Initialize ####

In [17]:
function tf {aws-vault exec terraform_user -- terraform $args}

function prompt_to_confirm { 
    $user_response = Read-host "Continue? y/n or 1/0"
    if ($user_response -ne "y" -and $user_response -ne 1) {break}
}

In [18]:
$aws_region = "us-east-2"
$env:AWS_REGION=$aws_region #for aws_vault
$env:TF_VAR_aws_region=$aws_region #for terraform

In [ ]:
(pwd).path

In [ ]:
terraform init #-upgrade

#### Workflow ####

In [ ]:
terraform fmt

In [19]:
terraform validate

Success! The configuration is valid.



In [ ]:
tf plan -out=tfplan
terraform show -json tfplan > plan.json

In [ ]:
#Output planned resource addresses and their names in the cloud

$plan = Get-Content -Raw -Path "plan.json" | ConvertFrom-Json

$resourcesToDeploy = $plan.resource_changes | Where-Object { 
    $_.change.actions -contains "create" -or $_.change.actions -contains "update" 
} | ForEach-Object { 
    "Type: " + $_.type
    "TF name: " + $_.name
    "Cloud name: " + $_.change.after.name
    "Name tag: " + $_.change.after.tags_all.name
    ""
}

$resourcesToDeploy
Get-Date

In [20]:
###      ###
### PLAN ###
###      ###

tf plan
Get-Date

data.aws_availability_zones.available: Reading...
module.vpc.aws_vpc.this[0]: Refreshing state... [id=vpc-0deaf554d8149cc66]
data.aws_availability_zones.available: Read complete after 1s [id=us-east-2]
module.vpc.aws_default_route_table.default[0]: Refreshing state... [id=rtb-0a5a93db1e4018fc0]
module.vpc.aws_route_table.private[1]: Refreshing state... [id=rtb-03a61d04945264052]
module.vpc.aws_default_security_group.this[0]: Refreshing state... [id=sg-002c2448065bba87d]
module.vpc.aws_subnet.private[0]: Refreshing state... [id=subnet-0885bccad6bef5df1]
module.vpc.aws_subnet.private[2]: Refreshing state... [id=subnet-089fd6c9ad7ecdd2e]
module.vpc.aws_subnet.private[1]: Refreshing state... [id=subnet-053639f36fbe824cb]
module.vpc.aws_default_network_acl.this[0]: Refreshing state... [id=acl-0ac42142e6664208e]
module.vpc.aws_route_table.private[2]: Refreshing state... [id=rtb-06bc091d46f3da1b1]
module.vpc.aws_route_table.private[0]: Refreshing state... [id=rtb-04619841f4db39717]
module.vpc

In [21]:
###       ###
### APPLY ###
###       ###

#try {prompt_to_confirm} catch {break}
tf apply -auto-approve
Get-Date

data.aws_availability_zones.available: Reading...
module.vpc.aws_vpc.this[0]: Refreshing state... [id=vpc-0deaf554d8149cc66]
data.aws_availability_zones.available: Read complete after 1s [id=us-east-2]
module.vpc.aws_default_route_table.default[0]: Refreshing state... [id=rtb-0a5a93db1e4018fc0]
module.vpc.aws_default_security_group.this[0]: Refreshing state... [id=sg-002c2448065bba87d]
module.vpc.aws_route_table.public[0]: Refreshing state... [id=rtb-046abfbbe0223ca87]
module.vpc.aws_default_network_acl.this[0]: Refreshing state... [id=acl-0ac42142e6664208e]
module.vpc.aws_internet_gateway.this[0]: Refreshing state... [id=igw-049b0e5e85b97d887]
module.vpc.aws_subnet.private[1]: Refreshing state... [id=subnet-053639f36fbe824cb]
module.vpc.aws_subnet.private[2]: Refreshing state... [id=subnet-089fd6c9ad7ecdd2e]
module.vpc.aws_subnet.private[0]: Refreshing state... [id=subnet-0885bccad6bef5df1]
module.vpc.aws_route_table.private[1]: Refreshing state... [id=rtb-03a61d04945264052]
module.vp

In [ ]:
###         ###
### DESTROY ###
###         ###

try {prompt_to_confirm} catch {break}
tf destroy -auto-approve
Get-Date

#### Info ####

In [ ]:
terraform version

In [ ]:
#Root module configuration
cat .\main.tf

In [ ]:
($resources = terraform state list) | Out-Host

In [ ]:
foreach ($resource in $resources) { terraform state show $resource }

In [ ]:
terraform output

#### Troubleshooting ####

In [ ]:
#$env:TF_LOG = "TRACE"
#$env:TF_LOG_PATH = "log.txt"
#$env:TF_LOG = ""

In [ ]:
tf apply -refresh-only -auto-approve